In [1]:
import pandas as pd
import numpy as np

In [2]:


# GameState
#     turn               # 1…9
#     active_player      # whose move
#     vp                 # victory‑points per player
#     resources          # {Money, Materials, Science} per player
#     map_tiles          # dict{hex_id → TileInfo}
#     fleets             # dict{player → list[Fleet]}
#     techs              # set of techs per player
#     action_discs_left  # num of actions each player can still take this round
#     explored_tiles     # list[TileNumber]  – needed by exploration sims
#     # … any other signals you care about (alliances, reputation, etc.)


In [3]:
## legal action generator

# function generate_actions(state: GameState, player):
#     actions = []
#     if state.action_discs_left[player] == 0:           return ['pass']

#     # 2.1 Explore
#     for neighbor_hex in unexplored_neighbors(player):
#         actions.append(Explore(neighbor_hex))

#     # 2.2 Build / Upgrade
#     if state.resources[player].Materials ≥ cost_to_build():
#         actions.append(Build(blueprint, location))

#     # 2.3 Move / Attack
#     for fleet in state.fleets[player]:
#         for target_hex in reachable_hexes(fleet):
#             actions.append(Move(fleet.id, target_hex))

#     # 2.4 Research
#     if state.resources[player].Science ≥ cheapest_available_tech():
#         actions.append(Research(tech_name))

#     return actions


In [4]:
## forward model

# function apply_action(state, action, sims_per_combat=300):
#     if action.type == 'Explore':
#         # 3a – draw ALL legal tiles for that sector
#         possible_tiles = draw_tiles(state, sector_of(action.hex))
#         # branch on each tile equally
#         return [(state_after_placing_tile(state, action.hex, tile),
#                  1/len(possible_tiles))
#                 for tile in possible_tiles]

#     if action.type == 'Move' and target_contains_enemy(state, action):
#         # 3b – combat branch: use your Monte‑Carlo wrapper
#         stats = run_monte_carlo(fleetA, fleetB, n=sims_per_combat)
#         p_win   = stats.win_count_A / sims_per_combat
#         p_lose  = stats.win_count_B / sims_per_combat
#         p_draw  = stats.draws        / sims_per_combat

#         return [
#             (post_combat_state(state, attacker_wins=True,  stats), p_win),
#             (post_combat_state(state, attacker_wins=False, stats), p_lose),
#             (post_combat_state(state, draw=True,           stats), p_draw)
#         ]

#     # 3c – all other actions are deterministic
#     return [(state_after_deterministic_change(state, action), 1.0)]


In [5]:
## evaluation function

# function heuristic(state, player):
#     vp_now     = state.vp[player]
#     vp_resources = (state.resources[player].Money   / 3) +   # rough trade‑off
#                    (state.resources[player].Science / 1) +   # easier to spend
#                    (state.resources[player].Materials / 2)

#     board_control = count_hexes_controlled(player) * 0.5
#     fleet_power   = total_fleet_cost(player) * 0.3

#     return vp_now + vp_resources + board_control + fleet_power


In [6]:
### search loop 

# function expectimax(state, player, depth, sims):
#     if depth == 0 or game_over(state):
#         return heuristic(state, player)

#     best = -∞
#     for action in generate_actions(state, player):
#         value = 0
#         for (next_state, p) in apply_action(state.copy(), action, sims):
#             value += p * expectimax(next_state,
#                                     next_player(player),
#                                     depth‑1,
#                                     sims)
#         best = max(best, value)
#     return best

In [ ]:
# best_score = -∞
# best_action = None
# for action in generate_actions(root_state, my_id):
#     score = 0
#     for (s′, p) in apply_action(root_state.copy(), action):
#         score += p * expectimax(s′, next_player(my_id), search_depth‑1, sims)
#     if score > best_score:
#         best_score, best_action = score, action

# return best_action


In [ ]:
# 6. Why this works for Eclipse
# Exploration randomness → handled in apply_action by branching on each tile.
# Combat variance → handled with your existing Monte‑Carlo wrapper.
# Alternating turns → the recursive next_player call flips the maximizer.
# High branching factor → cap depth (e.g., two full actions ahead) or switch to MCTS later; the plumbing above is compatible.

In [7]:
# 7. Places to plug in your existing notebooks
# Task	Drop‑in helper
# Tile expectations	expected_explore_values (for heuristic)
# Random tile draws	simulate_full_game (or simple .sample like above)
# Combat resolution	simulate_battle or run_monte_carlo inside apply_action
# Fleet power calc	reuse your blueprint dictionaries

In [ ]:
Next steps
Start with a micro‑game – 2 players, 4 hex map, no techs – and make sure the tree runs in < 1 sec at depth 2.
Add caching (transposition_table) so identical states aren’t re‑evaluated.
Once stable, swap expectimax for MCTS to look 4–5 moves ahead without exploding RAM.

